# Biclustering CC
Implementation proposed by Cheng & Church in Biclustering of Expression Data

In [1]:
%matplotlib inline
import numpy as np
from matplotlib import pyplot as plt
from sklearn.metrics import consensus_score

## MSR Definitions

In [2]:
%%latex
Define Mean Square Residue (MSR)$H(I,J)$:
$$H(I,J) = \frac{1}{|I||J|} \sum_{i \in I} \sum_{j \in J} (a_{ij} - a_{Ij} - a_{iJ} + a_{IJ})^2$$
where:
$$a_{iJ} = \frac{1}{|J|} \sum_{j \in J} a_{ij}$$
$$a_{Ij} = \frac{1}{|I|} \sum_{i \in I} a_{ij}$$
$$a_{IJ} = \frac{1}{|I||J|} \sum_{i \in I, j \in J} a_{ij}$$


<IPython.core.display.Latex object>

## MSR Computation

In [42]:
class MSR(object):
    def __init__(self,data):
        self.data=data
        self.n, self.m = data.shape
        self.aiJ = np.mean(data,axis=1)
        self.aIj = np.mean(data,axis=0)
        self.aIJ = np.mean(data)
        self._H = None
        self._HiJ = None
        self._HIj = None
    
    @property
    def H(self):
        if self._H is None:
            print ("computing MSR ...")
            self._H = self._compute_H()
            print ("MSR VALUE " + str(self._H))
        return self._H
        
    @property
    def HiJ(self):
        if self._HiJ is None:
            self._HiJ = self._compute_HiJ()
        return self._HiJ
    
    @property
    def HIj(self):
        if self._HIj is None:
            self._HIj = self._compute_HIj()
        return self._HIj
    
    def _compute_H(self):
        H = 0
        for i in range(self.n):
            for j in range(self.m):
                H  += (self.data[i,j] - self.aIj[j] - self.aiJ[i] + self.aIJ ) ** 2
        H *= 1.0/(self.n * self.m)       
        return H
    
    def _compute_HiJ(self):
        HiJ = np.zeros(self.n)
        for i in range(self.n):
            for j in range(self.m):
                HiJ[i] += ( self.data[i,j] - self.aIj[j] - self.aiJ[i] + self.aIJ )**2
        HiJ *= 1.0/self.m
        return HiJ

    def _compute_HIj(self):
        HIj = np.zeros(self.m)
        for j in range(self.m):
            for i in range(self.n):
                HIj[j] += ( self.data[i,j] - self.aIj[j] - self.aiJ[i] + self.aIJ )**2
        HIj *= 1.0/self.n
        return HIj


## Load Testing Data

In [5]:
import random
import gc
import pandas as pd
import glob
#data  = np.random.random((5, 5))
diFiles=glob.glob("../data/Di/*")
diFiles

['../data/Di/Jnk.csv',
 '../data/Di/CTNNB1.csv',
 '../data/Di/LEF1.csv',
 '../data/Di/Ikk2.csv',
 '../data/Di/Erk.csv',
 '../data/Di/MYC.csv',
 '../data/Di/IRF4.csv']

In [6]:
trainIndexFiles=glob.glob("../Train/*")
frames = list()
for f in trainIndexFiles:
    indices = pd.read_csv(f,sep=";")
    indexList=list(indices['0'])
    protein = f.split("/")[-1].split(".")[0]
    fpath = '../data/Di/'+f.split("/")[-1].split(".")[0]+".csv"
    diRead = pd.read_csv(fpath,sep=";")
    diPartialTrain=diRead[diRead.columns[5:]].loc[indexList].copy()
    diPartialTrain.insert(0,column='Protein',value =protein )
    del diRead
    gc.collect()
    frames.append(diPartialTrain)

In [7]:
diTrain = pd.concat(frames)
diTrain[diTrain.columns[1:]].head()

,X0min_neg_effect-X0min_neg_cause,X15min_effect-X0min_neg_cause,X15min_effect-X15min_cause,X30min_effect-X0min_neg_cause,X30min_effect-X15min_cause,X30min_effect-X30min_cause,X90min_effect-X0min_neg_cause,X90min_effect-X15min_cause,X90min_effect-X30min_cause,X90min_effect-X90min_cause,...,X240min_effect-X240min_cause,X360min_effect-X0min_neg_cause,X360min_effect-X15min_cause,X360min_effect-X30min_cause,X360min_effect-X90min_cause,X360min_effect-X120min_cause,X360min_effect-X180min_cause,X360min_effect-X210min_cause,X360min_effect-X240min_cause,X360min_effect-X360min_cause
20742,0.063368,1.281056,1.717385,-0.804117,-0.367788,-0.516333,0.602345,1.038674,0.890129,1.671097,...,-2.317443,0.513271,0.949600,0.801055,1.582023,0.515914,-0.552805,-1.103463,-1.242248,-0.662756
20743,0.063368,0.115374,0.357486,0.501125,0.743237,1.345916,1.024461,1.266573,1.869252,2.049611,...,-0.672834,-0.218719,0.023393,0.626072,0.806431,-0.026813,-0.159189,-0.266408,-0.571869,-0.609731
20744,1.682265,0.868625,1.684311,0.849711,1.665396,0.795722,0.378091,1.193777,0.324103,-1.269579,...,-2.784372,-1.791199,-0.975514,-1.845188,-3.438869,-3.124216,-3.172147,-2.151847,-3.780721,-4.990738
20746,1.965867,0.999337,-0.092929,0.830458,-0.261807,-0.590573,3.210072,2.117807,1.789041,1.833600,...,-4.073476,0.700332,-0.391934,-0.720699,-0.676141,-0.598271,-1.916351,-1.944383,-3.317719,-1.848127
20747,1.965867,5.096072,6.244664,1.793418,2.942010,1.103822,1.500028,2.648620,0.810432,0.723980,...,1.506882,0.592584,1.741176,-0.097012,-0.183463,-0.849972,-0.560518,-0.895341,0.620940,-0.904556


In [31]:
data = diTrain[diTrain.columns[1:]].as_matrix()[0:100]


In [32]:
msr = MSR(data)
print ("aIJ= " + str(msr.aIJ))
print ("H= " + str(msr.H))
print ("aiJ= " + str(msr.aiJ))
print ("aIj= " + str(msr.aIj))

aIJ= 0.233977724593
computing MSR ...
MSR VALUE 31.4782157563
H= 31.4782157563
aiJ= [-0.11337336  0.37507219 -0.47600993 -0.31639811  0.96364109 -0.30156956
  0.14792598  0.83027198 -0.54815898  0.98826394  0.07429657  0.59228038
 -0.41652465  1.32522594 -0.20167557  0.08988995 -0.44232465  0.54084251
 -0.47921131  1.04780054 -0.71248922  0.31107458 -0.20598849  0.39655691
 -0.42792283  1.04114841  0.52398097 -0.09109476  0.37143232  0.40867483
  0.37801423  0.65893614  0.52229227  0.27485395  0.44922896  0.25232749
  0.57618197  1.21852071 -0.1715021   0.93695677  0.11356463  0.97810524
  0.4560251   0.97664321 -0.59993298  0.81564055 -0.17194127  0.69987313
  0.19296214  1.15794653 -0.2196899   0.1790977   1.13582278  0.29699647
 -0.40464089  0.56622533  0.19457577 -0.01501339  0.60341288  0.42159929
 -0.5166737   0.23648974 -0.53505458  0.53843855 -1.00231531  1.40220681
 -1.10267953  1.05741305 -1.17982077  1.15136258  0.33948434  0.06949501
 -0.20365795 -0.19381633  0.64291017  1.

## Defining Algorithms

### Algorithm 0 - Remove unique nodes

In [33]:
def remove_unique_nodes(data, delta, I=None, J=None):
    it = 1
    
    if I is None:
        I = np.arange(len(data))
    
    if J is None:    
        J = np.arange(len(data[0]))
        
    while True:
        it += 1
        
        msr = MSR(data[I][:,J])
        
        if msr.H <delta:
            break
            
        if len(I) == 1 or len(J) == 1:
            break
        
        row_idx_to_remove = np.argmax(msr.HiJ)
        col_idx_to_remove = np.argmax(msr.HIj)
        
        if msr.HiJ[row_idx_to_remove] > msr.HIj[col_idx_to_remove]:
            # print("removing row " + str(row_idx_to_remove))
            # remove row if drops MSR
            I = np.delete(I,row_idx_to_remove)
            
        else:
            # print("removing col " + str(col_idx_to_remove))
            # remove col if drops MSR
            J = np.delete(J, col_idx_to_remove)
            
    return (data[I][:,J],I,J)
        
    

### Algorithm 1  - Remove multiple nodes

In [34]:
def remove_multiple_nodes(data, delta, alpha, I=None, J = None):
    it = 1
    if I is None:
        I = np.arange(len(data))
    
    if J is None:    
        J = np.arange(len(data[0]))
        
    removal_ocurred = True
    
    while True:
        it += 1
        
        msr = MSR(data[I][:,J]) 
        
        if msr.H <delta:
            break
        if len(I) == 1 or len(J) == 1:
            break
            
        row_idxs_to_remove = np.nonzero(msr.HiJ > alpha*msr.H)[0]
        
        if len(row_idxs_to_remove) > 0:
            # remove rows if there is some to remove
            I = np.delete(I, row_idxs_to_remove)
        else:
            # no removal ocurred
            removal_ocurred = False
            
        # recalcute MSR 
        msr = MSR(data[I][:,J])
        
        # remove if drops MSR
        cols_idxs_to_remove= np.nonzero(msr.HIj > alpha*msr.H)[0]
        
        if len(cols_idxs_to_remove) > 0:
            # remove coclumns if there is some to remove
            J = np.delete(J, cols_idxs_to_remove)
            
        else:
            # else remove unique nodes (algorithm 0)
            if not removal_ocurred:
                return remove_unique_nodes (data[I][:,J], delta, I, J)
    
    return (data[I][:,J],I,J)
        
        

### Algorithm 2 - Add Nodes

In [35]:
def add_nodes(data, orig_data, alpha, I, J):
    it = 1
    orig_n, orig_m = orig_data.shape
    orig_I = np.arange(orig_n)
    orig_J = np.arange(orig_m)
    I_not_in_bicluster = np.setdiff1d(orig_I, I)
    J_not_in_bicluster = np.setdiff1d(orig_J, J)
    addition_ocurred = True
    while addition_ocurred:
        # print ("%s iteration of node addition..." % it)
        it += 1
        addition_ocurred = False
        msr = MSR(orig_data[I][:,J])
        # column addition
        for j in J_not_in_bicluster:
            data_with_j = np.column_stack((data, orig_data[I][:,j]))
            msr_with_j = MSR(data_with_j)
            if msr_with_j.HIj[-1] <= msr.H:
                J = np.append(J, j)
                J_not_in_bicluster = np.delete(J_not_in_bicluster, np.where(j == J_not_in_bicluster)[0])
                data = data_with_j
                addition_ocurred = True
        msr = MSR(orig_data[I][:,J])
        # row addition
        for i in I_not_in_bicluster:
            data_with_i = np.row_stack((data, orig_data[i,J]))
            msr_with_i = MSR(data_with_i)
            if msr_with_i.HIj[-1] <= msr.H:
                I = np.append(I, i)
                I_not_in_bicluster = np.delete(I_not_in_bicluster, np.where(i == I_not_in_bicluster)[0])
                data = data_with_i
                adition_ocurred = True
        # inverted row addition
        msr = MSR(orig_data)
        rows_idxs_to_add_overlap = np.nonzero(msr.HIj <= alpha*msr.H)[0]
        rows_idxs_to_add = np.setdiff1d(rows_idxs_to_add_overlap, new_I)
        if len(rows_idxs_to_add) > 0:
            new_I = np.append(new_I, rows_idxs_to_add)
            inverted_rows = np.fliplr(orig_data[rows_idxs_to_add][:,J])
            data = np.row_stack((data, inverted_rows))
        else:
            addition_ocurred = False
    return (orig_data[I][:,J],I,J)


### Algorithm 3 - Find biclusters

In [36]:
def mask(data, I, J, min_v, max_v):
    for i in I:
        for j in J:
            data[i,j] = random.uniform(min_v, max_v)
    return data

In [38]:
def find_biclusters(data, delta, alpha, num_biclusters, num_iterations=1):
    biclusters = []

    min_v = np.min(data)
    max_v = np.max(data)

    for i in range(num_biclusters):
        n, m = data.shape
        # print (data.shape)

        # plt.matshow(data, cmap=plt.cm.Blues, vmin=min_v, vmax=max_v)
        plt.show()

        print ("--- Looking for bicluster %s:" % (i+1))

        
        if len(data) < 100 or len(data[0]) < 100:
            (C,I,J) = remove_unique_nodes(data, delta)
        else:
            (C,I,J) = remove_multiple_nodes(data, delta, alpha)

        (D,I,J) = add_nodes(C, data, alpha, I, J)

        biclusters.append(D.copy())

        plt.matshow(D, cmap=plt.cm.Blues, vmin=min_v, vmax=max_v)
        plt.show()

        A_line = mask(data, I, J, min_v, max_v)

    return biclusters

biclusters = find_biclusters(data, 1, 1, num_biclusters)

--- Looking for bicluster 1:
computing MSR ...
MSR VALUE 31.4782157563
computing MSR ...
MSR VALUE 30.5562748749
computing MSR ...
MSR VALUE 29.8615572173
computing MSR ...
MSR VALUE 29.2173077755
computing MSR ...
MSR VALUE 28.6041875463
computing MSR ...
MSR VALUE 28.0246051203
computing MSR ...
MSR VALUE 27.5331033262
computing MSR ...
MSR VALUE 27.0669422305
computing MSR ...
MSR VALUE 25.8726080719
computing MSR ...
MSR VALUE 24.6873600629
computing MSR ...
MSR VALUE 24.2500699266
computing MSR ...
MSR VALUE 23.8270006055
computing MSR ...
MSR VALUE 23.4074023374
computing MSR ...
MSR VALUE 23.0137328861
computing MSR ...
MSR VALUE 22.6280317158
computing MSR ...
MSR VALUE 22.2648131238
computing MSR ...
MSR VALUE 21.9182107206
computing MSR ...
MSR VALUE 21.5740861389
computing MSR ...
MSR VALUE 21.2274435346
computing MSR ...
MSR VALUE 20.8857022562
computing MSR ...
MSR VALUE 20.539319619
computing MSR ...
MSR VALUE 20.1837806633
computing MSR ...
MSR VALUE 19.8412293004
comput

UnboundLocalError: local variable 'new_I' referenced before assignment